In [7]:

import numpy as np
import pandas as pd 
import gc

In [3]:
x=pd.read_csv("train_mod.csv")
y=pd.read_csv("target_mod.csv")


In [3]:
pd.set_option('display.max_columns',None)

In [4]:
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler

In [5]:
rus=RandomUnderSampler(random_state=0)
X_sm,y_sm=rus.fit_resample(x,y)

In [8]:
del x
del y
gc.collect()

0

In [9]:
params={"n_estimators": [100,200, 300],
       "learning_rate": [0.05, 0.2, 1.0],
       "algorithm": ["SAMME","SAMME.R"],
       }

In [11]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import cross_val_score

grid_search = RandomizedSearchCV(AdaBoostClassifier(), params, n_jobs = -1, scoring='f1_micro', verbose =1)
grid_result = grid_search.fit(X_sm,y_sm.values.ravel())


Fitting 5 folds for each of 10 candidates, totalling 50 fits


In [15]:
print(grid_result.best_params_)   

{'n_estimators': 300, 'learning_rate': 0.2, 'algorithm': 'SAMME.R'}


In [16]:

adb = AdaBoostClassifier(n_estimators = 300, learning_rate = 0.2, algorithm= 'SAMME.R')
np.mean(cross_val_score(adb,X_sm,y_sm,scoring="f1_micro", cv =2))

/home/shux/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/shux/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.7560495579789637

In [17]:
adb.fit(X_sm, y_sm);

/home/shux/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [18]:
del X_sm
del y_sm
gc.collect()

1320

In [19]:
test=pd.read_csv("test_mod.csv")
ypred = adb.predict(test)
pd.DataFrame(ypred).to_csv('ada_pred2.csv')